In [ ]:
%pip install -r ../requirements.txt

In [2]:
import os
from typing import List, Dict, Tuple, Set, Optional, Union
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%%bash

if ! command -v kaggle &> /dev/null; then
    echo "kaggle is not installed\nInstalling kaggle..."
    pip install kaggle
fi

mkdir -p ~/.kaggle

if [ -f "kaggle.json" ]; then
    echo "kaggle.json found. Copying to ~/.kaggle/"
    cp kaggle.json ~/.kaggle/
    chmod 600 ~/.kaggle/kaggle.json
else
    echo "Error: kaggle.json not found in the current directory. Please upload it."
fi

In [4]:
%%bash

if [ ! -d "Training" ] || [ ! -d "Testing" ]; then
  echo "Dataset not found\nDownloading dataset..."
  kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset --unzip
fi

In [5]:
def get_class_paths(path: str):
  classes = []
  class_paths = []

  for label in os.listdir(path):
    label_path = os.path.join(path, label)

    if os.path.isdir(label_path):
      for image in os.listdir(label_path):
        image_path = os.path.join(label_path, image)

        classes.append(label)
        class_paths.append(image_path)

  df = pd.DataFrame({"class": classes, "path": class_paths})
  return df

In [6]:
tr_df = get_class_paths("Training")

In [ ]:
tr_df.head()

In [8]:
ts_df = get_class_paths("Testing")

In [ ]:
ts_df.head()

In [ ]:
plt.figure(figsize=(15, 7))
ax = sns.countplot(x="class", data=tr_df)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
plt.show()

In [ ]:
plt.figure(figsize=(15, 7))
ax = sns.countplot(x="class", data=ts_df)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
plt.show()

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from keras.src.layers import Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
valid_df, ts_df = train_test_split(ts_df, train_size=0.5, stratify=ts_df["class"])

In [ ]:
valid_df.head()

In [ ]:
ts_df.head()

In [16]:
batch_size = 32

img_size: Tuple[int, int] = (299, 299)

image_generator = ImageDataGenerator(
    rescale=1/255,
    brightness_range=(0.8, 1.2),
)

ts_gen = ImageDataGenerator(rescale=1/255)

In [ ]:
tr_gen = image_generator.flow_from_dataframe(
  tr_df,
  x_col="path",
  y_col="class",
  target_size=img_size,
  batch_size=batch_size,
  class_mode="categorical",
  subset="training"
)

valid_gen = image_generator.flow_from_dataframe(
  valid_df,
  x_col="path",
  y_col="class",
  target_size=img_size,
  batch_size=batch_size,
  class_mode="categorical"
)

ts_gen = ts_gen.flow_from_dataframe(
  ts_df,
  x_col="path",
  y_col="class",
  target_size=img_size,
  batch_size=batch_size,
  shuffle=False,
  class_mode="categorical"
)

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(16):
  plt.subplot(4, 4, i + 1)
  batch = next(tr_gen)
  image = batch[0][0]
  label = batch[1][0]
  plt.imshow(image)

  class_index = np.argmax(label)

  class_names = list(tr_gen.class_indices.keys())
  class_indicies = list(tr_gen.class_indices.values())

  index_position = class_indicies.index(class_index)

  class_name = class_names[index_position]

  plt.title(f'class: {class_name}')
  plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import tensorflow as tf

# Configure GPU memory growth before any other TensorFlow/Keras operations
print("TensorFlow version:", tf.__version__)
print("ROCm devices available:", tf.config.list_physical_devices('GPU'))

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
else:
    print("No GPU devices found")

#Now import Keras and perform other operations
import keras
keras.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
img_shape: Tuple[int, int, int] = (299, 299, 3)

base_model = tf.keras.applications.Xception(
  include_top=False,
  weights="imagenet",
  input_shape=img_shape,
  pooling="max"
)

input_tensor = Input(shape=img_shape)
processed_input = Lambda(lambda x: x)(input_tensor)
xception_output = base_model(processed_input)

model = Sequential([
  Input(shape=img_shape),
  base_model,
  Flatten(),
  Dropout(0.3),
  Dense(128, activation="relu"),
  Dropout(rate=0.25),
  Dense(4, activation="softmax")
])


In [ ]:
model.compile(
  Adamax(learning_rate=0.001),
  loss="categorical_crossentropy",
  metrics=["accuracy", Precision(), Recall()]
)

model.summary()


In [23]:
hist = model.fit(
    tr_gen,
    epochs=5,
    validation_data=valid_gen,
)

In [ ]:
hist.history

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

metrics = ["accuracy", "loss", "precision_1", "recall_1"]
tr_metrics = {m: hist.history[m] for m in metrics}
val_metrics = {m: hist.history[f"val_{m}"] for m in metrics}

best_epochs = {}
best_values = {}

for m in metrics:
    if m == 'loss':
        idx = np.argmin(val_metrics[m])
    else:
        idx = np.argmax(val_metrics[m])

    best_epochs[m] = idx + 1
    best_values[m] = val_metrics[m][idx]

plt.figure(figsize=(16, 12))
plt.style.use('fivethirtyeight')

for i, metric in enumerate(metrics):
    plt.subplot(2, 2, i+1)
    epochs = range(1, len(tr_metrics[metric]) + 1)

    plt.plot(
        epochs,
        tr_metrics[metric],
        'r-',
        label=f'Training {metric}'
    )
    plt.plot(
        epochs,
        val_metrics[metric],
        'g-',
        label=f'Validation {metric}'
    )
    plt.scatter(
        best_epochs[metric],
        best_values[metric],
        s=150,
        c='b',
        label=f'Best epoch = {best_epochs[metric]}'
    )

    title = f'{metric.replace("_1", "").capitalize()} Metric'
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel(metric.replace('_1', "").capitalize())
    plt.legend()
    plt.grid(True)

plt.suptitle('Model Training Metrics Over Epochs', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
train_score = model.evaluate(tr_gen, verbose=1)
valid_score = model.evaluate(valid_gen, verbose=1)
test_score = model.evaluate(ts_gen, verbose=1)

print(f"Train Accuracy: {train_score[1]*100:.2f}%")
print(f"Train Loss: {train_score[0]:.4f}")
print(f"\nValidation Accuracy: {valid_score[1]*100:.2f}%")
print(f"Validation Loss: {valid_score[0]:.4f}")
print(f"\n\nTest Accuracy: {test_score[1]*100:.2f}%")
print(f"Test Loss: {test_score[0]:.4f}")

In [ ]:
preds = model.predict(ts_gen)
y_pred = np.argmax(preds, axis=1)

class_dict = {
    0: "glioma",
    1: "meningioma",
    2: "no_tumor",
    3: "pituitary"
}

cm = confusion_matrix(ts_gen.classes, y_pred)
labels = list(class_dict.keys())
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=labels,
    yticklabels=labels
  )
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from PIL import Image

def predict(img_path: str) ->  None:
  labels = list(class_dict.keys())

  plt.figure(figsize=(6, 8))

  img = Image.open(img_path)
  resized_img = img.resize((299, 299))
  img_array = np.array(resized_img)
  img_array = np.expand_dims(img_array, axis=0) / 255.0

  predictions = model.predict(img_array)
  probabilities = list(predictions[0])

  predicted_class_idx = np.argmax(probabilities)
  predicted_class = labels[predicted_class_idx]

  plt.subplot(2, 1, 1)
  plt.imshow(resized_img)
  plt.title(f"Input MRI Image\nPredicted {predicted_class}")


  plt.subplot(2, 1, 2 )
  bars = plt.barh(labels, probabilities)
  plt.xlabel("Probability", fontsize=15)
  plt.title("Class Probabilities")

  ax = plt.gca()
  ax.bar_label(bars, fmt='%.2f')

  plt.tight_layout()
  plt.show()

  print(f"\nPredicted tumor types: {predicted_class}")

In [ ]:
predict("/content/Testing/glioma/Te-glTr_0000.jpg")

In [ ]:
model.save_weights("xception_model.weights.h5")

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras import regularizers

In [ ]:
batch_size = 16

img_size: Tuple[int, int] = (224, 224)

image_generator = ImageDataGenerator(
    rescale=1/255,
    brightness_range=(0.8, 1.2),
)

ts_gen = ImageDataGenerator(rescale=1/255)
tr_gen = image_generator.flow_from_dataframe(
  tr_df,
  x_col="path",
  y_col="class",
  target_size=img_size,
  batch_size=batch_size,
  class_mode="categorical",
  subset="training"
)

valid_gen = image_generator.flow_from_dataframe(
  valid_df,
  x_col="path",
  y_col="class",
  target_size=img_size,
  batch_size=batch_size,
  class_mode="categorical"
)

ts_gen = ts_gen.flow_from_dataframe(
  ts_df,
  x_col="path",
  y_col="class",
  target_size=img_size,
  batch_size=batch_size,
  shuffle=False,
  class_mode="categorical"
)

In [ ]:
cnn_model = Sequential()

cnn_model.add(Conv2D(256, (3, 3), padding='same', input_shape=(224, 224, 3), activation='relu'))
cnn_model.add(MaxPool2D(pool_size=(2, 2)))

cnn_model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
cnn_model.add(MaxPool2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
cnn_model.add(MaxPool2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
cnn_model.add(MaxPool2D(pool_size=(2, 2)))

cnn_model.add(Flatten())

cnn_model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Dense(4, activation='softmax'))

cnn_model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

cnn_model.summary()

In [ ]:
history = cnn_model.fit(tr_gen, epochs=5, validation_data=valid_gen)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

metrics = ["accuracy", "loss", "precision_1", "recall_1"]
tr_metrics = {m: history.history[m] for m in metrics}
val_metrics = {m: history.history[f"val_{m}"] for m in metrics}

best_epochs = {}
best_values = {}

for m in metrics:
    if m == 'loss':
        idx = np.argmin(val_metrics[m])
    else:
        idx = np.argmax(val_metrics[m])

    best_epochs[m] = idx + 1
    best_values[m] = val_metrics[m][idx]

plt.figure(figsize=(16, 12))
plt.style.use('fivethirtyeight')

for i, metric in enumerate(metrics):
    plt.subplot(2, 2, i+1)
    epochs = range(1, len(tr_metrics[metric]) + 1)

    plt.plot(
        epochs,
        tr_metrics[metric],
        'r-',
        label=f'Training {metric}'
    )
    plt.plot(
        epochs,
        val_metrics[metric],
        'g-',
        label=f'Validation {metric}'
    )
    plt.scatter(
        best_epochs[metric],
        best_values[metric],
        s=150,
        c='b',
        label=f'Best epoch = {best_epochs[metric]}'
    )

    title = f'{metric.replace("_1", "").capitalize()} Metric'
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel(metric.replace('_1', "").capitalize())
    plt.legend()
    plt.grid(True)

plt.suptitle('Model Training Metrics Over Epochs', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
train_score = cnn_model.evaluate(tr_gen, verbose=1)
valid_score = cnn_model.evaluate(valid_gen, verbose=1)
test_score = cnn_model.evaluate(ts_gen, verbose=1)

print(f"Train Accuracy: {train_score[1]*100:.2f}%")
print(f"Train Loss: {train_score[0]:.4f}")
print(f"\nValidation Accuracy: {valid_score[1]*100:.2f}%")
print(f"Validation Loss: {valid_score[0]:.4f}")
print(f"\n\nTest Accuracy: {test_score[1]*100:.2f}%")
print(f"Test Loss: {test_score[0]:.4f}")

In [ ]:
preds = cnn_model.predict(ts_gen)
y_pred = np.argmax(preds, axis=1)

class_dict = {
    0: "glioma",
    1: "meningioma",
    2: "no_tumor",
    3: "pituitary"
}

cm = confusion_matrix(ts_gen.classes, y_pred)
labels = list(class_dict.keys())
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=labels,
    yticklabels=labels
  )
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
clr = classification_report(ts_gen.classes, y_pred)
print(clr)

In [ ]:
cnn_model.save("cnn_model.h5")